In [ ]:
pip install mistralai

In [ ]:
import time
from mistralai import Mistral

# Configuration Mistral
MISTRAL_API_KEY = "Add your Key API"

client = Mistral(api_key=MISTRAL_API_KEY)
model = "mistral-large-latest"

def load_text_file(file_path):
    """Loads the contents of a text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

def load_questions(file_path):
    """Loads questions from a text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

def generate_prompt(base_prompt1, question,base_prompt2):
    """Generates a prompt combined with the question."""
    return f"{base_prompt1}\n\n{question}\n\n{base_prompt2}".strip()

def process_questions(questions, base_prompt1, base_prompt2):
    """Processes questions and returns results."""
    results = []

    for i, question in enumerate(questions, 1):
        retry_count = 0
        max_retries = 5
        initial_sleep_time = 2
         # Temps de sleep initial en secondes

        while retry_count < max_retries:
            try:
                prompt = generate_prompt(base_prompt1, question,base_prompt2)
                response = client.chat.complete(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0
                )

                content = response.choices[0].message.content
                results.append(f"Question {i}: {question}\n{content}\n")
                break  # Sortir de la boucle si la requête réussit

            except Exception as e:
                if hasattr(e, 'status_code') and e.status_code == 429:
                    retry_count += 1
                    sleep_time = initial_sleep_time * (2 ** (retry_count - 1))  # Temps de sleep progressif
                    print(f"Error 429: retry {retry_count} in {sleep_time} seconds...")
                    time.sleep(sleep_time)
                else:
                    results.append(f"Question {i}: {question}\nErreur: {str(e)}\n")
                    break  # Sortir de la boucle en cas d'autre erreur

        if retry_count == max_retries:
            results.append(f"Question {i}: {question}\nErreur: Maximum number of attempts reached.\n")

    return results

def save_results(results, filename):
    """Saves results in a text file."""
    with open(filename, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(result + "\n" + "-"*50 + "\n")

def main():
    base_prompt1 = load_text_file("prompt mistral1.txt")
    base_prompt2 = load_text_file("prompt mistral2.txt")
    questions = load_questions("questionnaire.txt")
    results = process_questions(questions, base_prompt1, base_prompt2)
    save_results(results, "gdpr_evaluation.txt")
    print(results)
    print("Evaluation completed. Results saved in gdpr_evaluation.txt")


if __name__ == "__main__":
    main()

In [ ]:
#stat by options
import re
from collections import Counter

def analyze_modeling_status(file_path):
    status_counts = Counter()

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Expression régulière pour capturer le statut de modélisation
    matches = re.findall(r'This question is:\s*(Fully modeled|Partially modeled|Not modeled|Should not be modeled)', content, re.IGNORECASE)

    # Comptabilisation des occurrences
    for match in matches:
        status_counts[match.lower()] += 1

    # Affichage des résultats
    for status, count in status_counts.items():
        print(f"{status.capitalize()}: {count}")

# Exemple d'utilisation
file_path = "gdpr_evaluation.txt"  # Remplacez par le chemin de votre fichier
analyze_modeling_status(file_path)